In [ ]:
"""libs"""
import re
import os
import pandas as pd
import sqlite3


br_conn = sqlite3.connect("./database/br_base_cnpj.db")
cadastrur_file = ("./database/CADASTUR_SIGA_SIMPLES_APPLE.xlsx")
# obteendo o caminho do diretório atual e construindo o caminho do arquivo a partir dele
current_dir = os.getcwd()

In [ ]:
# lendo os dados do cadastrur
cadastrur_dados = pd.read_excel(cadastrur_file, dtype='string')

In [ ]:
cadastrur_dados.rename(columns={'NúmerodeInscriçãodoCNPJ':'CNPJ', 'NomedaPessoaJurídica':"RAZAO_SOCIAL",
       'NomeFantasia':'NOME_FANTASIA', 'SituaçãoCadastral':'SITUACAO_CADASTRAL', 'SituaçãodaAtividade':'SITUACAO_ATIVIDADE',
       'TipodeEstabelecimento':"TIPO_ESTABELECIMENTO", 'NaturezaJurídica':'NATUREZA_JURIDICA', 'Porte':'PORTE',
       'EndereçoCompletoReceitaFederal':'ENDERECO_RFB', 'E - mailComercial':'EMAIL', 'Website':'SITE',
       'NúmerodoCertificado':'NUMERO_CERTIFICADO', 'ValidadedoCertificado':'VALIDADE_CERTIFICADO', 'Tipo':'TIPO', 'Especialidade':'ESPECIALIDADE',
       'Idiomas':'IDIOMAS', 'TelefoneComercial':'TELEFONE_COMERCIAL', 'Município':'MUNICIPIO',
       'EndereçoCompletoComercial':'ENDERECO_COMERCIAL', 'CNAE(S)relacionadosàatividade':'CNAE_ATIVIDADE',
       'StatusAssociadosAbrasel':'TIPO_ASSOCIADO'}, inplace=True)
cadastrur_dados = cadastrur_dados[["CNPJ", "RAZAO_SOCIAL", "NOME_FANTASIA", 
                                   "SITUACAO_CADASTRAL", "SITUACAO_ATIVIDADE", "TIPO_ESTABELECIMENTO", 
                                   "NATUREZA_JURIDICA", "PORTE", "ENDERECO_RFB", "EMAIL", "SITE", 
                                   "NUMERO_CERTIFICADO", "VALIDADE_CERTIFICADO",  "TIPO", "ESPECIALIDADE", 
                                   "IDIOMAS", "UF", "TELEFONE_COMERCIAL", "MUNICIPIO","ENDERECO_COMERCIAL", 
                                   "CNAE_ATIVIDADE", "TIPO_ASSOCIADO"]]
cadastrur_dados.head(1)

In [ ]:
# Verificar se a coluna SITE tem "@" e mover o valor para a coluna INSTAGRAM
cadastrur_dados.loc[cadastrur_dados["SITE"].str.contains("@"), "INSTAGRAM"] = cadastrur_dados["SITE"]
cadastrur_dados.loc[cadastrur_dados["SITE"].str.contains("@"), "SITE"] = None

# Verificar se a coluna SITE tem "facebook" e mover o valor para a coluna FACEBOOK
cadastrur_dados.loc[cadastrur_dados["SITE"].str.contains("facebook"), "FACEBOOK"] = cadastrur_dados["SITE"]
cadastrur_dados.loc[cadastrur_dados["SITE"].str.contains("facebook"), "SITE"] = None

# Mostrar o resultado final
cadastrur_dados.columns = cadastrur_dados.columns.str.strip()
print(cadastrur_dados[["SITE", "INSTAGRAM", "FACEBOOK"]])


In [ ]:
# lendo nossa base da receita
receita_dados = pd.read_sql_query("SELECT * FROM estabelecimentos", br_conn, dtype='string')

In [ ]:
# concatenando os dois dataframes
cadastrur_dados['CNPJ'] = cadastrur_dados['CNPJ'].astype('str')
receita_dados['CNPJ'] = receita_dados['CNPJ'].astype('str')
merge_dados_dados = cadastrur_dados.merge(receita_dados, how='left', on='CNPJ')

In [ ]:
merge_dados_dados.columns

In [ ]:
print(len(merge_dados_dados['CNPJ']))
merge_dados_dados.dropna(subset=['LATITUDE', 'LONGITUDE'],inplace=True)
merge_dados_dados.rename(columns={"RAZAO_SOCIAL_x":"RAZAO_SOCIAL", 'NOME_FANTASIA_x':'NOME_FANTASIA', 'SITUACAO_CADASTRAL_x':'SITUACAO_CADASTRAL','INSTAGRAM_x':'INSTAGRAM','FACEBOOK_x': 'FACEBOOK' }, inplace=True)
merge_dados_dados = merge_dados_dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'SITUACAO_CADASTRAL',
       'SITUACAO_ATIVIDADE', 'TIPO_ESTABELECIMENTO', 'SITE_x', 'NUMERO_CERTIFICADO', 
       'TIPO', 'ESPECIALIDADE', 'IDIOMAS', 'UF','TELEFONE_COMERCIAL',
       'CNAE_ATIVIDADE', 'TIPO_ASSOCIADO', 'ENDERECO_COMERCIAL', 'RUA', 'NUMERO', 'COMPLEMENTO', 'BAIRRO', 'CIDADE','CEP',
       'LATITUDE', 'LONGITUDE','SITE_y', 'CNAE_DESCRICAO','HORARIO_FUNCIONAMENTO', 'INSTAGRAM', 'FACEBOOK', 'OPCOES_DE_SERVICO']]
print(len(merge_dados_dados['CNPJ']))


In [ ]:
merge_dados_dados.columns

In [ ]:
# Importar o módulo re
import re

# Definir o padrão que combina os parênteses e usa um grupo de captura para o conteúdo
padrao = r"\((.*?)\)"

# Definir o código do país
codigo_pais = "+55"

# Definir a função que adiciona o código do país e remove os parênteses e os hífens
def formatar_numero(numero):
    # Remover os espaços antes e depois do número
    numero = numero.strip()

    # Remover os parênteses e capturar o conteúdo
    numero = re.sub(padrao, r"\1", numero)
    # Remover os hífens
    numero = numero.replace("-", "")
    # Adicionar o código do país no início
    numero = codigo_pais + numero
    # Retornar o número formatado
    return numero

# Aplica a função à coluna telefone do dataframe merge_dados_dados
merge_dados_dados['TELEFONE_COMERCIAL'] = merge_dados_dados['TELEFONE_COMERCIAL'].apply(formatar_numero)
merge_dados_dados['TELEFONE_COMERCIAL']

In [ ]:
# Inclui a coluna urls para busca no googlemaps
urls = []
for nome_fantasia, uf, cidade, bairro in merge_dados_dados[["NOME_FANTASIA", "UF", "CIDADE" ,"BAIRRO"]].itertuples(index=False):
    urls.append(f"https://www.google.com/maps/search/{nome_fantasia}+{uf}+{cidade}+{bairro}".replace(' ', '+'))

merge_dados_dados["URL"] = urls
# renomeia as colunas finais
merge_dados_dados.rename(columns={"CNPJ":"EIN (CNPJ)","RAZAO_SOCIAL":"CORPORATE_NAME",
                                  "NOME_FANTASIA":"TRADING NAME","RUA":"STREET","NUMERO":"ADDRESS_NUMBER",
                                  "COMPLEMENTO":"ADDRESS_COMPLEMENT","BAIRRO":"DISTRICT","CIDADE":"CITY","UF":"STATE",
                                  "CEP":"ZIP_CODE","TELEFONE_COMERCIAL":"PHONE_NUMBER", "SITE_x":"SITE","CNAE_DESCRICAO":"CATEGORY_(CNAE)",
                                  "HORARIO_FUNCIONAMENTO":"OPENING_HOURS","OPCOES_DE_SERVICO":"SERVICE_OPTIONS"}, inplace=True)


In [ ]:
merge_dados_dados.columns
# separa as colunas a serem salvas
merge_dados_dados = merge_dados_dados[['EIN (CNPJ)', 'TRADING NAME', 'CORPORATE_NAME','STREET', 'ADDRESS_NUMBER', 'ADDRESS_COMPLEMENT',
    'DISTRICT', 'CITY','STATE', 'ZIP_CODE', 'LATITUDE', 'LONGITUDE', 'PHONE_NUMBER', 'SITE', 'CATEGORY_(CNAE)', 'TIPO', 
    'ESPECIALIDADE', 'IDIOMAS', 'OPENING_HOURS', 'INSTAGRAM', 'FACEBOOK', 'SERVICE_OPTIONS', 'URL']]
merge_dados_dados

In [ ]:
# Seleciona as linhas de 0 a 4999 (inclusive) e todas as colunas
df_recorte = merge_dados_dados.iloc[0:4999, :]

# Converte o recorte em um arquivo json com o formato de registros
df_recorte.to_json("recorte_apple_abrasel.json", orient="records")
df_recorte.count()

In [ ]:
dados_apple = pd.read_json("recorte_apple_abrasel.json")
dados_apple.count()

In [1]:
import pandas as pd
dados = pd.read_csv('recorte_apple_abrasel_complemento.csv', sep=';', encoding='utf8')
dados.drop_duplicates(subset=['URL'], inplace=True)
dados.to_csv('recorte_apple_abrasel_complemento.csv', sep=';', encoding='utf8', index=False)
dados.count()

HORARIO_FUNCIONAMENTO    2433
OPCOES_DE_SERVICO        2351
URL                      4645
dtype: int64

In [ ]:

dados.count()

In [ ]:

dados_apple_merge = dados_apple.merge(dados, how='left', on='URL' )
dados_apple_merge.count()

In [ ]:
dados_apple_merge = dados_apple_merge[['EIN (CNPJ)', 'TRADING NAME', 'CORPORATE_NAME','STREET', 'ADDRESS_NUMBER', 'ADDRESS_COMPLEMENT',
    'DISTRICT', 'CITY','STATE', 'ZIP_CODE', 'LATITUDE', 'LONGITUDE', 'PHONE_NUMBER', 'SITE', 'CATEGORY_(CNAE)', 'TIPO', 
    'ESPECIALIDADE', 'IDIOMAS', 'HORARIO_FUNCIONAMENTO', 'INSTAGRAM', 'FACEBOOK', 'OPCOES_DE_SERVICO', 'URL']]
dados_apple_merge.count()

In [ ]:
dados_apple_merge.rename(columns={"OPCOES_DE_SERVICO":"SERVICE_OPTIONS","HORARIO_FUNCIONAMENTO":"OPENING_HOURS"}, inplace=True)
dados_apple_merge.count()

In [ ]:
dados_apple_merge = dados_apple_merge[['EIN (CNPJ)', 'TRADING NAME', 'CORPORATE_NAME','STREET', 'ADDRESS_NUMBER', 'ADDRESS_COMPLEMENT',
    'DISTRICT', 'CITY','STATE', 'ZIP_CODE', 'LATITUDE', 'LONGITUDE', 'PHONE_NUMBER', 'SITE', 'CATEGORY_(CNAE)', 'TIPO', 
    'ESPECIALIDADE', 'IDIOMAS', 'OPENING_HOURS', 'INSTAGRAM', 'FACEBOOK', 'SERVICE_OPTIONS']]
dados_apple_merge.count()

In [ ]:
dados_apple_merge.sort_values(by=['OPENING_HOURS','SERVICE_OPTIONS',"SITE"], inplace=True,
               ascending = [False, False, False], ignore_index=True)
dados_apple_merge

In [ ]:
# Define a função que adiciona o código do país e remove os parênteses e os hífens
def formatar_numero(numero):

    # Adicionar o código do país no início
    numero = '+' + str(numero)
    numero = numero.replace("+++++", "+")
    # Retornar o número formatado
    return numero

# Definir a função que adiciona zero À esquerda do cnpj
def zero_esquerda(cnpj):

    # preenche com zeros a esquerda
    cnpj = str(cnpj).zfill(14)
    
    # retorna o número formatado
    return cnpj

# Aplica a função à coluna telefone do dataframe merge_dados_dados
dados_apple_merge['PHONE_NUMBER'] = dados_apple_merge['PHONE_NUMBER'].apply(formatar_numero)
dados_apple_merge["EIN (CNPJ)"] = dados_apple_merge["EIN (CNPJ)"].apply(zero_esquerda)
dados_apple_merge = dados_apple_merge[['EIN (CNPJ)', 'TRADING NAME', 'CORPORATE_NAME','STREET', 'ADDRESS_NUMBER', 'ADDRESS_COMPLEMENT',
    'DISTRICT', 'CITY','STATE', 'ZIP_CODE', 'LATITUDE', 'LONGITUDE', 'PHONE_NUMBER', 'SITE', 'CATEGORY_(CNAE)', 'TIPO', 
    'ESPECIALIDADE', 'IDIOMAS', 'OPENING_HOURS', 'INSTAGRAM', 'FACEBOOK', 'SERVICE_OPTIONS']]
dados_apple_merge.dropna(subset=['OPENING_HOURS'], inplace=True)
dados_apple_merge.to_json("recorte_data_apple_abrasel.json", orient='records',indent=4)

dados_apple

In [ ]:
import pandas as pd

recorte = pd.read_json('./recorte_data_apple_abrasel.json', encoding='utf-8')
recorte.dropna(subset=['OPENING_HOURS'], inplace=True)
recorte = recorte[['EIN (CNPJ)', 'TRADING NAME', 'CORPORATE_NAME','STREET', 'ADDRESS_NUMBER', 'ADDRESS_COMPLEMENT',
    'DISTRICT', 'CITY','STATE', 'ZIP_CODE', 'LATITUDE', 'LONGITUDE', 'PHONE_NUMBER', 'SITE', 'CATEGORY_(CNAE)', 'OPENING_HOURS', 'INSTAGRAM', 'FACEBOOK', 'SERVICE_OPTIONS']]
recorte.to_json("data_apple_abrasel.json", orient='records',indent=4)
recorte

In [ ]:
print(dados.iloc[1:2, 0:1],sep=':')

In [ ]:
urls = []
for nome_fantasia, uf, cidade, bairro in merge_dados_dados[["NOME_FANTASIA", "UF", "CIDADE" ,"BAIRRO"]].itertuples(index=False):
    urls.append(f"https://www.google.com/maps/search/{nome_fantasia}+{uf}+{cidade}+{bairro}".replace(' ', '+'))
